In [1]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams as rc
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
rc['grid.linewidth']=0


In [9]:
# xyz=np.load('../../data/trajectory/sample_traj_Rouse_chain_T1.0_F4_Ta1_Esoft0_R020_Na1200_blocksize100_kb5.0_dt0.001_kr30.0_positions.npy')
# xyz=np.load('../../data/trajectory/sample_traj_Rouse_chain_T1.0_F0.3_Ta200.0_Esoft0_R020_Na1200_blocksize100_kb5.0_dt0.001_kr30.0_positions.npy')
# xyz=np.load('../../data/trajectory/sample_traj_Rouse_chain_T0.5_F0.1_Ta50.0_Esoft0_R0100_Na1200_blocksize100_kb5.0_dt0.001_kr30.0_positions.npy')
# xyz=np.load('../../data/trajectory/sample_traj_Rouse_chain_T0.5_F1.0_Ta10.0_Esoft0_R0100_Na1200_blocksize100_kb5.0_dt0.001_kr30.0_positions.npy')
# xyz=np.load('../../data/trajectory/sample_traj_Rouse_chain_T0.5_F0.0_Ta1.0_Esoft0_R0100_Na1200_blocksize100_kb5.0_dt0.001_kr30.0_positions.npy')
xyz=np.load('../../data/trajectory/traj_test_T300_F1.0_Ta10.0_Esoft0_R0100_Na1200_blocksize500_kb10.0_dt0.001_kr20.0_positions.npy')

print(xyz.shape)

(200, 1200, 3)


In [10]:
%matplotlib ipympl

fig = plt.figure()
# ax = fig.add_axes([0.,0.5,0.45,0.45],projection='3d')
ax = fig.add_axes([0,0,1,1],projection='3d')
# quiver = ax.quiver([],[],[],[])
#quiver = ax.quiver([],[],[],[],[],[])
N=1200

dr=4
Rx=xyz[:,:N,0].min()-dr, xyz[:,:N,0].max()+dr
Ry=xyz[:,:N,1].min()-dr, xyz[:,:N,1].max()+dr
Rz=xyz[:,:N,2].min()-dr, xyz[:,:N,2].max()+dr


R0=500
Rx=(-R0,R0)
Ry=(-R0,R0)
Rz=(-R0,R0)

# rcm=np.mean(np.mean(xyz[:,:N,:],axis=1),axis=0)
# print(rcm.shape)

zx=Rx[0]#rcm[0]
zy=Ry[1]#rcm[1]
zz=Rz[0]#rcm[2]

ax.scatter(xyz[0,:N,0],xyz[0,:N,1],xyz[0,:N,2],'.-',color='m')
# ax.plot(xyz[0,:N,0],xyz[0,:N,1], '.', zdir='z', zs=zz,color='b')
# ax.plot(xyz[0,:N,0],xyz[0,:N,2], '.', zdir='y', zs=zy,color='b')
# ax.plot(xyz[0,:N,1],xyz[0,:N,2], '.', zdir='x', zs=zx,color='b')

# ax.scatter(rcm[0],rcm[1],rcm[2], 'o',color='C3')

rval=np.linspace(-R0,R0,200)
# for val in [('x',zx,Rx),('y',zy,Ry),('z',zz,Rz)]:
#     ax.plot(rval,np.sqrt(R0**2-rval**2),'k-',zdir=val[0], zs=val[1])
#     ax.plot(rval,np.sqrt(R0**2-rval**2),'k-',zdir=val[0], zs=val[1])
#    ax.plot(rval,-np.sqrt(R0**2-rval**2),'k-',zdir=val[0], zs=val[1])

# aXY=fig.add_axes([0.5,0.1,0.3,0.3])
# aYZ=fig.add_axes([0.5,0.5,0.3,0.3])
# aXZ=fig.add_axes([0.1,0.1,0.3,0.3])
# ax.scatter(xyz[0,N:,0],xyz[0,N:,1],xyz[0,N:,2],'o',color='C1')
ax.set_xlim(Rx)
ax.set_ylim(Ry)
ax.set_zlim(Rz)

ax.view_init(40,29)
# fig.tight_layout()
def update(ii):
    fig.clear()
    # ax = fig.add_axes([0.,0.5,0.45,0.45],projection='3d')
    # aXY=fig.add_axes([0.5,0.1,0.3,0.3])
    # aYZ=fig.add_axes([0.5,0.5,0.3,0.3])
    # aXZ=fig.add_axes([0.1,0.1,0.3,0.3])
    # # ax = fig.add_axes([0,0,1,1],)
    ax = fig.add_axes([0,0,1,1],projection='3d')

    p=xyz[ii]
    # print(p.shape)
    # line.set_data(p[:,0],p[:,1])
    # line.set_3d_properties(p[:,2])
    # plt.draw()
    # ax.plot(p[10:13,0],p[10:13,1],p[10:13,2],'o',color='C0',alpha=1)
    # ax.scatter(p[:N,0],p[:N,1],p[:N,2],'.-',color='C0',alpha=1)
    # ax.plot(p[:N,0],p[:N,1],p[:N,2],'.-',color='C0',alpha=1)
    # ax.scatter(p[N:,0],p[N:,1],p[N:,2],'.-',color='C1',alpha=1)
    # ax.scatter(p[:N,0],p[:N,1],p[:N,2], 'o', zdir='z', offset=-10,color='C1')
    # aXZ.plot(p[:N,0],p[:N,2],'o',color='C0')
    # aYZ.plot(p[:N,1],p[:N,2],'o',color='C0')

    # ax.plot(p[300:303,0],p[300:303,1],p[300:303,2],'o',color='C1',alpha=1)
    # ax.scatter(p[N:,0],p[N:,1],p[N:,2],'o',color='C1',alpha=0.5)
    # ax.plot(masked_xyz[:100,0],masked_xyz[:100,1],'go',markersize=3)
    
    # ax.plot(p[:N,0],p[:N,1], '.-', zdir='z', zs=zz,color='C1')
    # ax.plot(p[:N,0],p[:N,2], '.-', zdir='y', zs=zy,color='C1')
    # ax.plot(p[:N,1],p[:N,2], '.-', zdir='x', zs=zx,color='C1')

    for val in [('x',zx,'b', (1,2)),('y',zy,'b',(0,2)),('z',zz,'b',(0,1))]:
        ax.plot(p[:N,val[3][0]],p[:N,val[3][1]], '.', zdir=val[0], zs=val[1],color='0.8', alpha=0.5)

        ax.plot(rval,np.sqrt(R0**2-rval**2),'k-',zdir=val[0], zs=val[1],lw=0.5)#,c=val[2])
        ax.plot(rval,-np.sqrt(R0**2-rval**2),'k-',zdir=val[0], zs=val[1],lw=0.5)#, c=val[2])
    
    # for cc,chrm in zip(['C0','C1','C2','C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10','C11' ],range(12)):
    #     ci,cf=100*chrm,100*(chrm+1)
    #     ax.plot(p[ci:cf,0],p[ci:cf,1],p[ci:cf,2],'.',color=cc)
    ax.plot(p[:N,0],p[:N,1],p[:N,2],'.',color='m')

    ax.set_zlim(Rz)
    ax.set_xlim(Rx)
    ax.set_ylim(Ry)

    # for axi in [aXY,aXZ,aYZ]:
    # # axX.set_zlim(-R0, R0)
    #     axi.plot(rval,np.sqrt(R0**2-rval**2),'k-')
    #     axi.plot(rval,-np.sqrt(R0**2-rval**2),'k-')
    #     axi.set_xlim(-R0, R0)
    #     axi.set_ylim(-R0, R0)
    

    # return line,

# update(10)
ani = FuncAnimation(fig, update, frames=np.arange(1,200,1), interval=200, blit=True,repeat=False)

# ani.save('ani_RC100_T0p5_F1_Ta10_kb5_R0100_full.gif', writer='pillow')
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
poly1=xyz[0,:100,:]
hull=ConvexHull(poly1)

In [17]:
fig=plt.figure()
axi=fig.add_axes([0,0,0.8,0.8],projection='3d')

for simplex in hull.simplices:
    tri = Poly3DCollection(poly1[simplex])
    tri.set_color('b')
    tri.set_alpha(0.5)
    tri.set_edgecolor('k')
    axi.add_collection3d(tri)
axi.scatter(poly1[:,0],poly1[:,1],poly1[:,2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [58]:
def autocorrFFT(x):
    N=len(x)
    F = np.fft.fft(x,n=2*N)  #2*N because of zero-padding
    PSD = F * F.conjugate()
    res = np.fft.ifft(PSD)
    res= (res[:N]).real   
    # n=N*np.ones(N)-np.arange(0,N) #divide res(m) by (N-m)
    return res/(N-np.arange(0,N)) #this is the autocorrelation in convention A

#r is an (T,3) ndarray: [time stamps,dof]
def msd_fft(r):
    N=len(r)
    D=np.square(r).sum(axis=1)
    D=np.append(D,0)
    S2=sum([autocorrFFT(r[:, i]) for i in range(r.shape[1])])
    Q=2*D.sum()
    S1=[]
    for m in range(N):
        Q=Q-D[m-1]-D[N-m]
        S1.append(Q/(N-m))
    return np.array(S1) - 2*S2

In [59]:
d1=msd_fft(xyz[:20,0,:])
print(d1)

[ 0.          1.6366861   2.85522743  3.55156537  4.25256764  4.94496162
  5.17026927  5.90420385  7.01804853  8.98931395 10.2192749  11.9409824
 13.08291883 14.40406205 15.46976339 17.71912129 18.91299827 16.3943899
 13.35601067 12.57748312]


In [47]:
d2=msd_fft(xyz[:10,0,:])

10 (10,) (10,) (10,)
10 (10,) (10,) (10,)
10 (10,) (10,) (10,)


In [65]:
c=np.array([1,2,3])
d=np.tile(c,(100,1,1))
d.shape

(100, 1, 3)